In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount
from utils.contract_retrievers import retrieve_farm_by_address
from utils.utils_chain import get_token_details_for_address, get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken
from contracts.pair_contract import PairContract
from contracts.farm_contract import FarmContract, ClaimRewardsFarmEvent, EnterFarmEvent
from contracts.dex_proxy_contract import DexProxyContract, DexProxyClaimRewardsEvent, DexProxyEnterFarmEvent, DexProxyExitFarmEvent, DexProxyRemoveLiquidityEvent, DexProxyAddLiquidityEvent
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

context = Context()

In [ ]:
farm_contract: FarmContract
farm_contract = context.get_contracts(config.FARMS_V2)[0]
proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
pair_contract: PairContract
pair_contract = context.get_contracts(config.PAIRS_V2)[0]
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

address setup

In [ ]:
user = Account(pem_file="~/Documents/sh1.pem")
user.address = Address("erd1zf3w4gxjry72me8mz0q3gue8av8wep7l2x7ave5jaelhsx98sxlsuhe729")
user.sync_nonce(context.network_provider.proxy)

claim proxy farm rewards

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyClaimRewardsEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.claim_rewards_proxy(user, context.network_provider.proxy, event)

exit position

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyExitFarmEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount // 2)
txhash = proxy_contract.exit_farm_proxy(user, context.network_provider.proxy, event)

enter position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount,
                               proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

exit lp position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No tokens")
    exit()
event = DexProxyRemoveLiquidityEvent(pair_contract, farming_token_amount, farming_token_nonce, 1, 1)
txhash = proxy_contract.remove_liquidity_proxy(user, context.network_provider.proxy, event)

create lp position

In [ ]:
first_token_nonce, first_token_amount, _ = get_token_details_for_address(pair_contract.firstToken, user.address.bech32(), context.network_provider.proxy)

xmex = next(locked_token for locked_token in proxy_contract.locked_tokens if "XMEX" in locked_token)
xmex_list = get_all_token_nonces_details_for_account(xmex, user.address.bech32(), context.network_provider.proxy)
biggest_xmex = {}
for xmex_found in xmex_list:
    # get the highest amount one in the list
    if int(xmex_found['balance']) > int(biggest_xmex.get('balance', 0)):
        biggest_xmex = xmex_found
second_token_nonce, second_token_amount = biggest_xmex['nonce'], int(biggest_xmex['balance'])

if first_token_amount == 0 or second_token_amount == 0:
    print(f"No tokens: first token amount: {first_token_amount}; second token amount {second_token_amount}")
    exit()
event = DexProxyAddLiquidityEvent(pair_contract, pair_contract.firstToken, first_token_nonce, first_token_amount, 1,
                                  xmex, second_token_nonce, second_token_amount, 1,)
txhash = proxy_contract.add_liquidity_proxy(user, context.network_provider.proxy, event)

exit position

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyExitFarmEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.exit_farm_proxy(user, context.network_provider.proxy, event)

Lock tokens

In [ ]:
_, mex_amount, _ = get_token_details_for_address(proxy_contract.token, user.address.bech32(), context.network_provider.proxy)

lockable_tokens = ESDTToken(proxy_contract.token, 0, mex_amount)
txhash = energy_contract.lock_tokens(user, context.network_provider.proxy, [[lockable_tokens], 720])

create lp position

In [ ]:
first_token_nonce, first_token_amount, _ = get_token_details_for_address(pair_contract.firstToken, user.address.bech32(), context.network_provider.proxy)

xmex = next(locked_token for locked_token in proxy_contract.locked_tokens if "XMEX" in locked_token)
xmex_list = get_all_token_nonces_details_for_account(xmex, user.address.bech32(), context.network_provider.proxy)
biggest_xmex = {}
for xmex_found in xmex_list:
    # get a specific xmex nonce from the list
    if int(xmex_found['nonce']) == 17:
        biggest_xmex = xmex_found
second_token_nonce, second_token_amount = biggest_xmex['nonce'], int(biggest_xmex['balance'])

if first_token_amount == 0 or second_token_amount == 0:
    print(f"No tokens: first token amount: {first_token_amount}; second token amount {second_token_amount}")
    exit()
event = DexProxyAddLiquidityEvent(pair_contract, pair_contract.firstToken, first_token_nonce, first_token_amount, 1,
                                  xmex, second_token_nonce, second_token_amount, 1,)
txhash = proxy_contract.add_liquidity_proxy(user, context.network_provider.proxy, event)

enter new position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount // 2,
                               "", 0, 0)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

merge position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount,
                               proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

increase proxy lp energy

In [ ]:
from utils.utils_chain import dec_to_padded_hex
epochs_increase = 360

farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
if farming_token_nonce == 0:
    print("No tokens")
    exit()

print(f"User has {farming_token_amount} farming tokens with nonce {dec_to_padded_hex(farming_token_nonce)}")
txhash = proxy_contract.increase_proxy_lp_token_energy(user, context.network_provider.proxy, 
                                                       [[ESDTToken(proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount)],
                                                        epochs_increase])

In [ ]:
epochs_increase = 360

txhash = proxy_contract.increase_proxy_lp_token_energy(user, context.network_provider.proxy, 
                                                       [[ESDTToken("LKLP-03a2fa", 625343, 44304399799466962)],
                                                        epochs_increase])

increase proxy farm energy

In [ ]:
from utils.utils_chain import dec_to_padded_hex
epochs_increase = 1440

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No tokens")
    exit()

print(f"User has {farm_token_amount} tokens with nonce {dec_to_padded_hex(farm_token_nonce)}")
txhash = proxy_contract.increase_proxy_farm_token_energy(user, context.network_provider.proxy, 
                                                       [[ESDTToken(proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount//2)],
                                                        epochs_increase])

Decode XMEXFARM

In [ ]:
from multiversx_sdk_network_providers import ApiNetworkProvider

token_nonce = "3c92"

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
print(proxy_contract.get_all_decoded_farm_token_attributes_from_network(mainnet_api, int(token_nonce, 16)))

Decode XMEXLP

In [ ]:
from multiversx_sdk_network_providers import ApiNetworkProvider

token_nonce = "2173"

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
print(proxy_contract.get_all_decoded_lp_token_attributes_from_network(mainnet_api, int(token_nonce, 16)))

XFARM Transaction checker

In [ ]:
from utils.utils_chain import dec_to_padded_hex

tx_hash = "af414491da7f4deefe81cead9936d5cb06b32110d8401867cb7481befa051fd5"
mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
used_api = context.network_provider.api

# Get the transaction details
tx = context.network_provider.api.get_transaction(tx_hash)
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

# Get transaction inputs
sender = tx.sender.bech32()
payment = tx.raw_response['action']['arguments']['transfers'][0]['identifier']
payment_value = tx.raw_response['action']['arguments']['transfers'][0]['value']

xmexfarm_decode, xmexlp_decode, lktoken_decode = proxy_contract.get_all_decoded_farm_token_attributes_from_network(used_api, 
                                                                                                                   int(payment.split("-")[2], 16))

rated_lp_tokens = int(xmexfarm_decode.get('proxy_token_amount') * (int(payment_value) / xmexfarm_decode.get('farm_token_amount')))
rated_loked_tokens = int(xmexlp_decode.get('locked_tokens_amount') * (int(payment_value) / xmexfarm_decode.get('farm_token_amount')) * (xmexfarm_decode.get('proxy_token_amount') / xmexlp_decode.get('lp_token_amount')))

expected_ops = [
    { # User transfer the farm token to the proxy contract
        "action": "transfer",
        "sender": sender,
        "receiver": proxy_contract.address,
        "identifier": payment,
        "value": str(payment_value)
    },
    { # Proxy contract burns the original proxy farm token
        "action": "burn",
        "sender": proxy_contract.address,
        "receiver": sender,
        "identifier": payment,
        "value": str(payment_value)
    },
    { # Proxy contract burns the original proxy lp token
        "action": "burn",
        "sender": proxy_contract.address,
        "receiver": sender,
        "identifier": f"{xmexfarm_decode.get('proxy_token_id')}-{dec_to_padded_hex(xmexfarm_decode.get('proxy_token_nonce'))}",
        "value": str(rated_lp_tokens)
    },
    { # Proxy contract transfers the original xmex to energy contract
        "action": "transfer",
        "sender": proxy_contract.address,
        "receiver": energy_contract.address,
        "identifier": f"{xmexlp_decode.get('locked_tokens_id')}-{dec_to_padded_hex(xmexlp_decode.get('locked_tokens_nonce'))}",
        "value": str(rated_loked_tokens)
    },
    { # Energy contract burns the original xmex
        "action": "burn",
        "sender": energy_contract.address,
        "receiver": sender,
        "identifier": f"{xmexlp_decode.get('locked_tokens_id')}-{dec_to_padded_hex(xmexlp_decode.get('locked_tokens_nonce'))}",
        "value": str(rated_loked_tokens)
    },
    { # Energy contract adds new xmex with increased energy
        "action": "addQuantity",
        "sender": energy_contract.address,
        "receiver": sender,
        "collection": xmexlp_decode.get('locked_tokens_id'),
        "value": str(rated_loked_tokens)
    },
    { # Energy contract transfers the new xmex to the proxy contract
        "action": "transfer",
        "sender": energy_contract.address,
        "receiver": proxy_contract.address,
        "collection": xmexlp_decode.get('locked_tokens_id'),
        "value": str(rated_loked_tokens)
    },
    { # Proxy contract creates new proxy lp token
        "action": "create",
        "sender": proxy_contract.address,
        "receiver": sender,
        "collection": proxy_contract.proxy_lp_token,
        "value": str(rated_lp_tokens)
    },
    { # Proxy contract creates the new proxy farm token
        "action": "create",
        "sender": proxy_contract.address,
        "receiver": sender,
        "collection": proxy_contract.proxy_farm_token,
        "value": str(payment_value)
    },
    { # Transfer the new proxy farm token to user
        "action": "transfer",
        "sender": proxy_contract.address,
        "receiver": sender,
        "collection": proxy_contract.proxy_farm_token,
        "value": str(payment_value)
    },
]

# Get and check transaction operations
ops = tx.raw_response['operations']

assert(len(ops) == len(expected_ops))

# Take each operation and match it with the ones in expected ops. Try to match only the fields expected in each expected ops. 
# Operations are unordered. If all operations match, the transaction is valid.
for op in ops:
    found = False
    matched = True
    # print(op)
    for expected_op in expected_ops:
        # print(f'Matching with {expected_op}')
        if all(op.get(key) == expected_op.get(key) for key in expected_op.keys()):
            found = True
            expected_ops.remove(expected_op)
            break
    if not found:
        print(f"Operation not matched {op}")
# List remaining operations line by line
for op in expected_ops:
    print(f"Remaining operation: {op}")